Recipe Hunter

Run all and input information when requested.

In [64]:
import requests
import pandas as pd
from pprint import pprint
import textwrap
import os
from pathlib import Path

# Get notebook current folder path
notebook_dir = Path().absolute()  

# Concatenate with .env file name
env_path = notebook_dir / 'recipehunter.env'  

with open(env_path) as f:
    env_vars = f.readlines()
    
API_KEY = str([x.strip().split('=')[-1] for x in env_vars if x.startswith('API_KEY')][0])

#print(API_KEY)

In [65]:
# List of ingredients to search for recipes 
#ingredients = ['pasta', 'broccoli', 'onion']
ingredients = input("Enter ingredients separated by comma: ").split(", ")
ingr_str = '%2C'.join(ingredients)

In [66]:
#List ingredients to avoid
#avoid = ['tomato','milk','cheese']
avoid = input("Enter ingredients to avoid: ").split(", ")
avoid_str = "%2C".join(avoid)

In [67]:
# Check for common ingredients
common_ings = set(ingredients).intersection(avoid)
if common_ings:
    print(f"Ingredients {', '.join(common_ings)} appear in both lists. Please revise inputs.")
    exit()

In [68]:
# Spoonacular recipe search API URL
api_url = f'https://api.spoonacular.com/recipes/findByIngredients?ingredients={ingr_str}&apiKey={API_KEY}'  
#api_url = f'https://api.spoonacular.com/recipes/findByIngredients?ingredients={ingr_str}&avoidIngredients={avoid_str}&apiKey={API_KEY}'  


# Make request to API
res = requests.get(api_url)
print(res)
data = res.json()
recipes_df = pd.DataFrame(data)
#recipes_df.head()

<Response [200]>


In [69]:
# Filter recipes with ingredients we want to avoid
for avoid_ingr in avoid:
    recipes_df = recipes_df[~recipes_df['missedIngredients'].apply(lambda x: avoid_ingr in str(x))]

filtered_recipes = []
for recipe in data:
    if not any(avoid_ingr in recipe['missedIngredients'] for avoid_ingr in avoid):
        filtered_recipes.append(recipe)

#for recipe in filtered_recipes:
#    print(recipe['title'])
#    print(f"Used Ingredients: {recipe['usedIngredientCount']}") 
#    print(f"Missed Ingredients: {recipe['missedIngredientCount']}")
#    print('-'*40)

#print("Filtered Recipes:")  
#recipes_df.head()

In [70]:
# Code to get filtered_recipes list 

print("Filtered Recipes:")
for i, r in enumerate(filtered_recipes):
    print(f"{i+1}. {r['title']}")
    
choice = int(input("Enter recipe number to get details: "))
chosen_recipe = filtered_recipes[choice-1]

url2 = f"https://api.spoonacular.com/recipes/{chosen_recipe['id']}/information?apiKey={API_KEY}"
res2 = requests.get(url2)
data2 = res2.json()

print(f"\n{chosen_recipe['title']}")
print("Full Ingredients:")
print(", ".join([item['name'] for item in data2['extendedIngredients']]))

url3 = f"https://api.spoonacular.com/recipes/{chosen_recipe['id']}/summary?apiKey={API_KEY}"
res3 = requests.get(url3)
data3 = res3.json()

print(f"\nSummary:\n")

# Set width limit 
lim = 120  

# Wrap text  
wrapped_summary = textwrap.fill(data3['summary'], width=lim)

# Print wrapped text
print(wrapped_summary)

print(data2['sourceUrl'])

Filtered Recipes:
1. Crispy Buttermilk Fried Chicken
2. Zucchini Chicken Omelette
3. Lemon and Garlic Slow Roasted Chicken
4. Slow Cooker Rosemary Whole Chicken
5. All Day Simple Slow-Cooker FALL OFF the BONE Ribs
6. Easy Chicken Pot Pie
7. Skillet Roasted Chicken & Potatoes
8. Chicken Suya
9. Buffalo Chicken Wings Wonton Wraps
10. Turbo Chicken

Zucchini Chicken Omelette
Full Ingredients:
eggs, water, zucchini, salt and pepper, oil, milanese chicken left over

Summary:

Zucchini Chicken Omelette might be a good recipe to expand your main course recipe box. This recipe serves 2. Watching
your figure? This gluten free, dairy free, fodmap friendly, and whole 30 recipe has <b>210 calories</b>, <b>13g of
protein</b>, and <b>16g of fat</b> per serving. For <b>72 cents per serving</b>, this recipe <b>covers 11%</b> of your
daily requirements of vitamins and minerals. This recipe from Foodista has 6 fans. If you have milanese chicken left
over, oil, salt and pepper, and a few other ingredient